In [3]:
import os
import cv2
from ultralytics import YOLO
import supervision as sv
from tqdm import tqdm

# --- 1. CONFIGURATION ---

# Path to the folder containing your custom trained YOLOv8 model weights.
# The script will look for a 'best.pt' file inside this directory.
MODEL_WEIGHTS_PATH = "./runs/detect/yolov8n_car_parts_finetune/weights/best.pt"

# Path to the directory containing the videos you want to process.
VIDEO_SOURCE_DIR = "/mnt/c/Users/mohsinali.mirza/Downloads/Grounding/videos_to_process/"

# Path to the directory where the annotated videos will be saved.
VIDEO_OUTPUT_DIR = "/mnt/c/Users/mohsinali.mirza/Downloads/Grounding/videos_annotated/"

# Your class names, in the exact same order as defined in your dataset.yaml file.
# This list MUST match the one used during training.
CLASSES = ["unit", "speaker", "motor", "assembly"]

# --- 2. SCRIPT SETUP ---

# Create the output directory if it doesn't exist
os.makedirs(VIDEO_OUTPUT_DIR, exist_ok=True)
print(f"Annotated videos will be saved to: {VIDEO_OUTPUT_DIR}")

# Load the custom YOLOv8 model
print(f"Loading model from: {MODEL_WEIGHTS_PATH}")
try:
    model = YOLO(MODEL_WEIGHTS_PATH)
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure the path to your 'best.pt' file is correct.")
    exit()

# Setup Supervision annotators for drawing boxes and labels
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Find all video files in the source directory
try:
    video_files = [f for f in os.listdir(VIDEO_SOURCE_DIR) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
    if not video_files:
        print(f"No video files found in {VIDEO_SOURCE_DIR}. Exiting.")
        exit()
except FileNotFoundError:
    print(f"Error: The directory was not found: {VIDEO_SOURCE_DIR}. Please check the path.")
    exit()

print(f"Found {len(video_files)} video(s) to process.")

# --- 3. VIDEO PROCESSING LOOP ---
for video_filename in video_files:
    source_video_path = os.path.join(VIDEO_SOURCE_DIR, video_filename)
    output_video_path = os.path.join(VIDEO_OUTPUT_DIR, video_filename)

    print(f"\nProcessing video: {video_filename}")

    try:
        # Use Supervision's VideoInfo to get video properties
        video_info = sv.VideoInfo.from_video_path(source_video_path)
        total_frames = video_info.total_frames
        
        # Use Supervision's get_video_frames_generator to iterate through the video
        frame_generator = sv.get_video_frames_generator(source_path=source_video_path)
        
        # Setup video writer to save the annotated video
        with sv.VideoSink(output_video_path, video_info) as sink:
            # Create a progress bar for the current video
            with tqdm(total=total_frames, desc=f"Annotating {video_filename}") as pbar:
                for frame in frame_generator:
                    # Run YOLOv8 inference on the frame
                    results = model(frame)[0]
                    
                    # Convert YOLOv8 results to Supervision's Detections object
                    detections = sv.Detections.from_ultralytics(results)
                    
                    # --- MODIFICATION: Filter detections based on a confidence threshold ---
                    # Create a boolean mask for detections with confidence > 0.8
                    confidence_mask = detections.confidence > 0.8
                    # Apply the mask to filter the detections
                    filtered_detections = detections[confidence_mask]
                    
                    # Create labels with class name AND confidence score for the filtered detections
                    labels = [
                        f"{CLASSES[class_id]} {confidence:.2f}"
                        for class_id, confidence
                        in zip(filtered_detections.class_id, filtered_detections.confidence)
                    ]
                    
                    # Annotate the frame with bounding boxes and the new labels using the filtered detections
                    annotated_frame = box_annotator.annotate(
                        scene=frame.copy(), 
                        detections=filtered_detections
                    )
                    annotated_frame = label_annotator.annotate(
                        scene=annotated_frame, 
                        detections=filtered_detections, 
                        labels=labels
                    )
                    
                    # Write the annotated frame to the output video file
                    sink.write_frame(annotated_frame)
                    pbar.update(1)

    except Exception as e:
        print(f"An error occurred while processing {video_filename}: {e}")

print("\n--- All videos have been processed successfully! ---")



Annotated videos will be saved to: /mnt/c/Users/mohsinali.mirza/Downloads/Grounding/videos_annotated/
Loading model from: ./runs/detect/yolov8n_car_parts_finetune/weights/best.pt
Found 4 video(s) to process.

Processing video: 20250624_185423.mp4


Annotating 20250624_185423.mp4:   0%|          | 0/266 [00:00<?, ?it/s]


0: 384x640 (no detections), 20.6ms
Speed: 3.3ms preprocess, 20.6ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)


Annotating 20250624_185423.mp4:   0%|          | 1/266 [00:00<01:34,  2.81it/s]


0: 384x640 (no detections), 18.9ms
Speed: 2.4ms preprocess, 18.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 30.5ms
Speed: 2.4ms preprocess, 30.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.1ms
Speed: 2.5ms preprocess, 15.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)


...


0: 384x640 1 loud speaker, 20.0ms
Speed: 2.3ms preprocess, 20.0ms inference, 5.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 loud speaker, 7.9ms
Speed: 2.0ms preprocess, 7.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 loud speaker, 19.3ms
Speed: 1.8ms preprocess, 19.3ms inference, 4.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 loud speaker, 7.6ms
Speed: 2.2ms preprocess, 7.6ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)


Annotating 20250624_185500.mp4:  99%|█████████▊| 138/140 [00:04<00:00, 33.69it/s]


0: 384x640 1 loud speaker, 7.9ms
Speed: 2.1ms preprocess, 7.9ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 loud speaker, 8.0ms
Speed: 2.2ms preprocess, 8.0ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)


Annotating 20250624_185500.mp4: 100%|██████████| 140/140 [00:04<00:00, 32.84it/s]


--- All videos have been processed successfully! ---
